In [1]:
from network import *
import numpy as np
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

num_bag_train = 2000
bags_list_train = []
labels_list_train = []
for i in range(num_bag_train):
    indices = random_state.randint(0, len(train_x), bag_length)
    bags_list_train.append(train_x[indices])
    labels_list_train.append(train_y[indices])
train_bag_x = np.array(bags_list_train)
train_bag_y = np.array(labels_list_train)
np.save('train_x', train_bag_x)
np.save('train_y', train_bag_y)
print(train_bag_y.mean())
# 测试集
num_bag_test = 500
bags_list_test = []
labels_list_test = []
for i in range(num_bag_test):
    indices = random_state.randint(0, len(test_x), bag_length)
    bags_list_test.append(test_x[indices])
    labels_list_test.append(test_y[indices])
test_bag_x = np.array(bags_list_test)
test_bag_y = np.array(labels_list_test)
np.save('test_x', test_bag_x)
np.save('test_y', test_bag_y)
print(test_bag_y.mean())

def label_bags(label_array):
    counts = np.bincount(label_array)
    if counts[7] and counts[9]:
        return 1
    else:
        return 0
    
def generate_data(target_number=9, num_bag_train=2000, num_bag_test=1000, bag_size=10, seed=1):
    (train_images, train_labels), (test_images, test_labels) \
        = tf.keras.datasets.mnist.load_data()
    train_images, train_labels = train_images[:, :,
                                              :, np.newaxis] / 255.0, train_labels
    test_images, test_labels = test_images[:, :,
                                           :, np.newaxis] / 255.0, test_labels
    random_state = np.random.RandomState(seed)
    bags_list_train = []
    labels_list_train = []
    for i in range(num_bag_train):
        indices = random_state.randint(0, 60000, bag_size)
        labels_in_bag = train_labels[indices]
        bags_list_train.append(train_images[indices])
        labels_list_train.append(labels_in_bag)
    bags_list_test = []
    labels_list_test = []
    for i in range(num_bag_test):
        indices = random_state.randint(0, 10000, bag_size)
        labels_in_bag = test_labels[indices]
        bags_list_test.append(test_images[indices])
        labels_list_test.append(labels_in_bag)
    return 


In [2]:
train_data, test_data = generate_data()

In [3]:
feature_extract_layer = tf.keras.Sequential([
    tf.keras.Input(shape=(28,28,1)),
    tf.keras.layers.Conv2D(20, kernel_size=5, activation='relu'),
    tf.keras.layers.MaxPool2D(strides=2),
    tf.keras.layers.Conv2D(50, kernel_size=5, activation='relu'),
    tf.keras.layers.MaxPool2D(strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation='relu')
])
attention_layer = Attention(500,128,1)
classifer_layer = tf.keras.layers.Dense(1, activation='sigmoid')
my_model = MIL_attention(feature_extract_layer, attention_layer, classifer_layer)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

In [6]:
train(train_data, my_model, 1, optimizer)

Time for epoch 1 is 5.743231773376465 sec
Loss: 0.08956658840179443
Accuracy: 0.976


In [9]:
pred = [my_model(i[0]) for i in test_data]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [14]:
true = [i[1] for i in test_data]

In [38]:
pred_label = [1 if i[0][0] > 0.5 else 0 for i in pred]

In [31]:
(np.array(pred_label) == np.array(true)).sum()/1000

0.939

In [40]:
pred[2]

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9795852]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[9.9136995e-04],
        [1.6952099e-05],
        [1.7990903e-03],
        [1.4925064e-03],
        [4.3216706e-04],
        [9.9475241e-01],
        [3.1526511e-06],
        [7.9765567e-07],
        [1.5405336e-04],
        [3.5752100e-04]], dtype=float32)>)

In [42]:
test_data[2][2]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [45]:
pred[5]

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.98300296]], dtype=float32)>,
 <tf.Tensor: shape=(11, 1), dtype=float32, numpy=
 array([[1.4789340e-04],
        [1.0431281e-03],
        [1.3995325e-04],
        [8.4589956e-06],
        [9.4913272e-04],
        [7.3591148e-04],
        [2.6245814e-04],
        [9.9467802e-01],
        [1.8532244e-05],
        [1.0638222e-03],
        [9.5277268e-04]], dtype=float32)>)

In [46]:
test_data[5][2]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [33]:
train_data_x = [i[0] for i in train_data]

In [34]:
train_data_y = [i[2] for i in train_data]

In [37]:
train_data_set_x = np.concatenate(train_data_x)

In [26]:
train_data_set_y = np.concatenate(train_data_y)

In [30]:
np.save('x.npy', train_data_set_x)
np.save('y.npy', train_data_set_y)

In [32]:
my_model.feature_extract.save('feature_extract.h5')